In [1]:
import opensim
import traceback
import pandas as pd
from connections import AWS
import xml.etree.ElementTree as ET
from biomech.opensim import load_osim_model

$\textbf{OpenSim: Joint Reaction Analysis}$

- Dynamically load and update subject models/files to run tool
- Store results using `AWS` connector

In [2]:
# load a .mot file (e.g., results from an IK run)
def load_mot_file(path: str, skip_rows: int = 10) -> pd.DataFrame:
    return pd.read_csv(path, sep='\s+', skiprows=skip_rows)

In [68]:
""" INITIALIZE AWS CONNECTION """
aws_connection = AWS()
aws_connection.connect()

[AWS]: Port 5433 is free.
[AWS]: Connected to RDS endpoint.


$\textit{JRA Setup}$

- Pre-set file paths, constants, etc
- Reference and load AWS objects/files

In [4]:
from jra_functions import *

In [5]:
""" OPENSIM SETUP """
# set paths
JRA_PATH = 'setup_jra.xml'
MODEL_PATH = 'trial_model.osim'
IK_PATH = 'trial_motion.mot'
ID_PATH = 'trial_moments.sto'

# set constants
STATES_IN_DEGREES = True            # default input file -- if not in header, uses this.
REMOVE_SPHERES = True               # delete spheres as force elements (tbd if used)

# set opensim settings
opensim.Logger.setLevelString('error')

# other parameters (TBD)
Qds = []
contact_spheres = {}                # should stay none <-- no GRFs

In [ ]:
""" AWS SETUP """
# load all subject info, ball release times
subject_info = aws_connection.load_subject_info()
ball_release_times = aws_connection.load_s3_object('biomechanics/subjects/summary/ball_release_times.csv')

# load all (filtered) IK files
s3_objects = aws_connection.list_s3_objects(prefix='biomechanics/subjects/')
mot_files = [obj for obj in s3_objects if obj.endswith('.mot') and '_ik' in obj]
id_files = [obj for obj in s3_objects if obj.endswith('.sto') and '_id' in obj]

# set XML dir path 
XML_DIR = 'biomechanics/xml_templates'

In [ ]:
# initialize error log
error_log = []

# iterate through subjects
for subject in subject_info['subject_id'].unique():
    
    try:
        # get subject info (ID -- string; throwing hand)
        subject_id = str(subject)
        throwing_hand = subject_info[subject_info['subject_id'] == subject]['throws'].values[0]

        # list all subject trials (in trc_processed) & iterate through
        subject_trials_ik = [t for t in mot_files if f'{subject_id}' in t]
        subject_trials_id = [s for s in id_files if f'{subject_id}' in s]

         # create jra folder if it doesn't exist
        aws_connection.create_s3_folder(f'biomechanics/subjects/{subject_id}/jra/')
        
        # check if subject has trials
        if len(subject_trials_ik) > 0:
            print(f'Processing subject {subject_id} with {len(subject_trials_ik)} trials...')

            
            """ FILE LOADING & SETUP """
            # load subject caled model (w/ hand-ball segment updates) to use for trials
                # NOTE: this model was used for ID
            scaled_model_xml = aws_connection.load_xml_from_s3(
                f'biomechanics/subjects/{subject_id}/osim/{subject_id}_scaled_model_hb.xml'
            )
            scaled_model_tree = ET.ElementTree(scaled_model_xml)
            scaled_model_tree.write(f'trial_model.osim')                # write model locally

            # load subject model for trials
            scaled_model = opensim.Model(f'trial_model.osim')

            # handle force set
                # NOTE: force set is not used, but this matches steps from reference code
            scaled_model, force_set = handle_force_set(scaled_model, REMOVE_SPHERES)
            
            # iterate through trials here for IK/ID relevant tasks
            for trial in subject_trials_ik:
                trial_id = trial.split('/')[-1].split('_ik.mot')[0]                     # get trial ID for saving/checking for ID

                """ IK SETUP """
                # load IK trial from S3 --> trial_motion.mot
                aws_connection.s3.download_file(
                    aws_connection.bucket_name,
                    trial,
                    'trial_motion.mot'
                )
                trial_mot_data = pd.read_csv(IK_PATH, sep='\s+', skiprows=10)           # for JRA prep below
                
                # handle coordinates & update kinematic states
                coords, n_coords, coord_names, controller_set = handle_coordinates(scaled_model)
                q, qd, state_time, in_degrees, state_names = update_kinematic_states(
                    n_coords,
                    coords,
                    coord_names,
                    ik_path=IK_PATH,
                    preset_degree_state=STATES_IN_DEGREES,
                )

                """ ID SETUP """
                # load ID trial from S3
                matching_id_file = next((item for item in subject_trials_id if trial_id in item), None)         # look for matching ID file
                if matching_id_file:
                    # download from S3 --> trial_moments.sto
                    aws_connection.s3.download_file(
                        aws_connection.bucket_name,
                        matching_id_file,
                        'trial_moments.sto'
                    )

                    # load into opensim format
                    id_table = opensim.TimeSeriesTable(ID_PATH)
                    id_time = id_table.getIndependentColumn()                           # get time column from ID results    
                
                else:
                    print(f'No matching ID file found for trial {trial_id}. Skipping ID processing.')
                    # log error
                    error_log.append({
                        'subject_id': subject_id,
                        'error': f'No matching ID file found for trial {trial_id}. Skipping ID processing.'
                    })
                    
                    continue

                """ MODEL JRA PREP """
                jra_setup = setup_joint_reaction_analysis(
                    model=scaled_model,
                    coord_names=coord_names,
                    mot_data=trial_mot_data,
                    jra_path=JRA_PATH,
                    print_to_xml=False
                )
                jointReaction_finished = run_joint_reaction_analysis(
                    scaled_model,
                    jra_setup['state'],
                    jra_setup['jointReaction'],
                    coords,
                    n_coords,
                    controller_set,
                    id_table,
                    id_time,
                    state_time,
                    q,
                    qd,
                    jra_setup['system_position_idxs'],
                    jra_setup['system_velocity_idxs']
                )
                
                """ POSTPROCESSING """
                # includes the following:
                    # print to local dir
                    # evt extraction
                    # AWS upload
                    # NOTE: need to extract ball release times for all subjects
                jointReaction_finished.printResults('trial_results')
                trial_jra_results = pd.read_csv('trial_results_JointReactionAnalysis_ReactionLoads.sto', sep='\s+', skiprows=11)

                # extract EVT data
                evt_col = get_evt_col(throwing_hand)
                evt_data = trial_jra_results[['time', evt_col]].rename(columns={evt_col: 'elbow_varus_torque'})

                # postprocess EVT data (filter to FC->BR, normalize time)
                br_time = ball_release_times[ball_release_times['study_id'] == trial_id]['ball_release_time'].values[0]
                evt_data_clean = postprocess_evt_results(evt_data, ball_release_time=br_time)

                # AWS upload (full results & EVT data)
                aws_connection.upload_to_s3(
                    trial_jra_results,
                    f'biomechanics/subjects/{subject_id}/jra/{trial_id}_jra_results.sto',
                )
                aws_connection.upload_to_s3(
                    evt_data_clean,
                    f'biomechanics/subjects/{subject_id}/jra/{trial_id}_jra_evt.sto',
                )

    # error handling
    except Exception as e:
        print(f"Error processing subject {subject_id}: {e}")

        traceback.print_exc()

        # log error
        error_log.append({
            'subject_id': subject_id,
            'error': str(e)
        })

        continue



[AWS]: Folder s3://pitch-ml/biomechanics/subjects/2609/jra/ already exists.
Processing subject 2609 with 19 trials...
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/jra/2609_01_jra_results.sto
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/jra/2609_01_jra_evt.sto
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/jra/2609_02_jra_results.sto
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/jra/2609_02_jra_evt.sto
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/jra/2609_03_jra_results.sto
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/jra/2609_03_jra_evt.sto


In [8]:
# concatenate error log
error_log_df = pd.DataFrame(error_log)

# write error log to S3 bucket
aws_connection.upload_to_s3(
    error_log_df.to_csv(index=False),
    'subjects/summary/error_jra_log.csv'
)

[AWS]: Uploaded object to s3://pitch-ml/subjects/summary/error_jra_log.csv


$\textbf{Close AWS Connection}$

In [67]:
aws_connection.close()

[AWS]: Database connection closed.
[AWS]: SSH tunnel stopped.
